In [32]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import datetime
import time
import re
import pytz
import requests
import tempfile


driver = webdriver.Firefox(executable_path= r'/home/ezequiel/Desktop/geckodriver')

# tela de login
driver.get('https://sip.prefeitura.sp.gov.br/sip/login.php?sigla_orgao_sistema=PMSP&sigla_sistema=SEI')
driver.find_element_by_id('txtUsuario').send_keys('d858414')
time.sleep(0.1)
driver.find_element_by_id('pwdSenha').send_keys('013451Ab')
select  = Select(driver.find_element_by_id('selOrgao'))
select.select_by_value('11')
driver.find_element_by_id('sbmLogin').click()

In [30]:
s = requests.session()
while True:

    if IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).exists():
        
        driver.get('https://sei.prefeitura.sp.gov.br/sei/controlador.php?acao=procedimento_controlar&acao_origem=principal&acao_retorno=principal&inicializando=1&infra_sistema=100000100&infra_unidade_atual=110001902&infra_hash=d0200d7767665961283eec748cb3b4891ea63b713cb4c1ab0bd0573e894de4ff')
        print('cadastrando')

        m = IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).prefetch_related('informacao',
                                                                                       'parte_citada',
                                                                                       'parte_oposta',
                                                                                       'intimacao',
                                                                                       'unidade'
                                                                                      ).first()
        documento = Documentos.objects.get(intimacao=m.intimacao)
        r = s.get(documento.link_doc, verify=False)

        temp_int = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False)
        with temp_int as file:
            file.write(r.content)
        documento = temp_int.name


        # iniciar processo
        driver.find_element_by_xpath('//*[@id="main-menu"]/li[3]/a').click()
        # Comunicação de decisão judicial e pedido de informação
        driver.find_element_by_xpath('//*[@tabindex="1003"]').click()
        # Parametros da tela
        driver.find_element_by_id('txtDescricao').send_keys(m.informacao.processo)

        # Assunto
        driver.find_element_by_id('imgPesquisarAssuntos').click()
        time.sleep(1)
        driver.switch_to_window(driver.window_handles[1])
        driver.find_element_by_id('txtPalavrasPesquisaAssuntos').send_keys('adm civ')
        driver.find_element_by_id('btnPesquisar').click()
        driver.find_element_by_xpath('//*[@id="lnkInfraT-1325"]/img').click()
        driver.switch_to_window(driver.window_handles[0])

        # Interessado

        driver.find_element_by_id('txtInteressadoProcedimento').send_keys(m.parte_oposta.nome)
        driver.find_element_by_id('txtInteressadoProcedimento').send_keys(Keys.ENTER)
        try:
            alert = driver.switch_to.alert
            alert.accept()
        except:
            pass

        # doc restrito
        driver.find_element_by_id('optRestrito').click()
        while 'Documento Preparatório' not in driver.page_source:
            time.sleep(1)
        driver.find_element_by_id('btnSalvar').click()

        # pagina seguinte
        driver.switch_to_frame(driver.find_element_by_id('ifrArvore'))
        id_procedimento = re.search('(?<=id_procedimento=)\d+', driver.current_url).group()
        m.sei_gerado = driver.find_element_by_id(f'span{id_procedimento}').text

        # inserir documento
        driver.switch_to_default_content()
        driver.switch_to_frame(driver.find_element_by_id('ifrVisualizacao'))
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="divArvoreAcoes"]/a[1]/img').click()
        driver.find_element_by_xpath('//*[@id="tblSeries"]/tbody/tr[1]/td/a[2]').click()


        # # mandado de citação
        select = Select(driver.find_element_by_id('selSerie'))
        select.select_by_value('2188')
        # # data do mandado de citação
        driver.find_element_by_id('txtDataElaboracao').send_keys(datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Brazil/East')), '%d/%m/%Y'))
        # # nato-digital
        driver.find_element_by_id('lblNato').click()
        # # restrito
        driver.find_element_by_id('lblRestrito').click()

        driver.find_element_by_id('filArquivo').send_keys(documento)

        driver.find_element_by_id('btnSalvar').click()


        driver.find_element_by_xpath('//*[@id="span22966958"]').click()
    else:
        time.sleep(10)

cadastrando


/home/ezequiel/pgm/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/ezequiel/pgm/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: use driver.switch_to.window instead


NoSuchElementException: Message: Unable to locate element: [id="txtPalavrasPesquisaAssuntos"]


In [24]:
driver = webdriver.Firefox(executable_path=r'/home/ezequiel/Desktop/geckodriver')